In [ ]:
!pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.7/192.7 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 1024 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.50.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

In [ ]:
from datasets import Dataset
from datasets import load_dataset


ds = load_dataset("shubhamtr/llama2_diet_planner")


README.md:   0%|          | 0.00/658 [00:00<?, ?B/s]

diet.txt:   0%|          | 0.00/1.49M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
train_data = ds["train"]


In [ ]:
def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=2048)

tokenized_dataset = train_data.map(tokenize)
tokenized_dataset.set_format(type="torch")


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./diet-plan-model",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    save_steps=500,
    logging_steps=100,
    fp16=True,  # Set True if using GPU that supports float16
    save_total_limit=2,
)


In [ ]:

model = FastLanguageModel.get_peft_model(
    model,
    r=16,                 # LoRA rank
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none"
)

Unsloth 2025.3.19 patched 28 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [ ]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=tokenized_dataset,
)


In [ ]:
trainer.train()


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,000 | Num Epochs = 3 | Total steps = 1,500
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 1 x 1) = 2
 "-____-"     Trainable parameters = 24,313,856/3,000,000,000 (0.81% trained)


Step,Training Loss
100,0.174600
200,0.091000
300,0.079400
400,0.078500
500,0.073200
600,0.065700
700,0.067900
800,0.063500
900,0.064000
1000,0.065200


In [ ]:
trainer.state


TrainerState(epoch=3.0, global_step=1500, max_steps=1500, logging_steps=100, eval_steps=500, save_steps=500, train_batch_size=2, num_train_epochs=3, num_input_tokens_seen=0, total_flos=1.085796478007378e+17, log_history=[{'loss': 0.1746, 'grad_norm': 0.7086471319198608, 'learning_rate': 4.666666666666667e-05, 'epoch': 0.2, 'step': 100}, {'loss': 0.091, 'grad_norm': 0.43106943368911743, 'learning_rate': 4.3333333333333334e-05, 'epoch': 0.4, 'step': 200}, {'loss': 0.0794, 'grad_norm': 0.5552883148193359, 'learning_rate': 4e-05, 'epoch': 0.6, 'step': 300}, {'loss': 0.0785, 'grad_norm': 0.7198107838630676, 'learning_rate': 3.6666666666666666e-05, 'epoch': 0.8, 'step': 400}, {'loss': 0.0732, 'grad_norm': 0.29127684235572815, 'learning_rate': 3.3333333333333335e-05, 'epoch': 1.0, 'step': 500}, {'loss': 0.0657, 'grad_norm': 0.36995771527290344, 'learning_rate': 3e-05, 'epoch': 1.2, 'step': 600}, {'loss': 0.0679, 'grad_norm': 0.27374526858329773, 'learning_rate': 2.6666666666666667e-05, 'epoch

In [ ]:
model.save_pretrained("llama3_diet_model")
tokenizer.save_pretrained("llama3_diet_model")


('llama3_diet_model/tokenizer_config.json',
 'llama3_diet_model/special_tokens_map.json',
 'llama3_diet_model/tokenizer.json')

In [ ]:
from huggingface_hub import login, create_repo, upload_folder

login()  # Paste your Hugging Face token here



In [ ]:

create_repo("llama3-diet-model", private=True)  #  False if you want public



RepoUrl('https://huggingface.co/shubhamtr/llama3-diet-model', endpoint='https://huggingface.co', repo_type='model', repo_id='shubhamtr/llama3-diet-model')

In [ ]:
# Upload the model and tokenizer
upload_folder(
    repo_id="shubhamtr/llama3-diet-model",  # <--- your actual username
    folder_path="llama3_diet_model",
    path_in_repo=".",
)

  0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/97.3M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/shubhamtr/llama3-diet-model/commit/b68b2546855d328f261da6c1dca218261efb8cc0', commit_message='Upload folder using huggingface_hub', commit_description='', oid='b68b2546855d328f261da6c1dca218261efb8cc0', pr_url=None, repo_url=RepoUrl('https://huggingface.co/shubhamtr/llama3-diet-model', endpoint='https://huggingface.co', repo_type='model', repo_id='shubhamtr/llama3-diet-model'), pr_revision=None, pr_num=None)